In [1]:
from datascience import *
path_data = '../../../assets/data/'
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import numpy as np

# Unindo Tabelas por Colunas
Frequentemente, dados sobre os mesmos indivíduos são mantidos em mais de uma tabela. Por exemplo, um escritório universitário pode ter dados sobre o tempo de conclusão do curso de cada aluno, enquanto outro tem dados sobre a mensalidade e a ajuda financeira do aluno.

Para entender a experiência dos alunos, pode ser útil unir os dois conjuntos de dados. Se os dados estiverem em duas tabelas, cada uma com uma linha por aluno, então gostaríamos de juntar as colunas, garantindo que as linhas sejam correspondidas para que as informações de cada aluno permaneçam em uma única linha.

Vamos fazer isso no contexto de um exemplo simples e depois usar o método com um conjunto de dados maior.

A tabela `cones` é uma que encontramos anteriormente. Agora, suponha que cada sabor de sorvete tenha uma classificação que esteja em uma tabela separada.

In [2]:
cones = Table().with_columns(
    'Flavor', make_array('strawberry', 'vanilla', 'chocolate', 'strawberry', 'chocolate'),
    'Price', make_array(3.55, 4.75, 6.55, 5.25, 5.75)
)
cones

Flavor,Price
strawberry,3.55
vanilla,4.75
chocolate,6.55
strawberry,5.25
chocolate,5.75


In [3]:
ratings = Table().with_columns(
    'Kind', make_array('strawberry', 'chocolate', 'vanilla'),
    'Stars', make_array(2.5, 3.5, 4)
)
ratings

Kind,Stars
strawberry,2.5
chocolate,3.5
vanilla,4


Cada uma das tabelas tem uma coluna que contém os sabores de sorvete: `cones` tem a coluna `Flavor`, e `ratings` tem a coluna `Kind`. As entradas nessas colunas podem ser usadas para vincular as duas tabelas.

O método `join` cria uma nova tabela na qual cada cone na tabela `cones` é aumentado com as informações de Estrelas na tabela `ratings`. Para cada cone em `cones`, `join` encontra uma linha em `ratings` cujo `Kind` corresponde ao `Flavor` do cone. Temos que dizer ao `join` para usar essas colunas para fazer a correspondência.

In [4]:
rated = cones.join('Flavor', ratings, 'Kind')
rated

Flavor,Price,Stars
chocolate,6.55,3.5
chocolate,5.75,3.5
strawberry,3.55,2.5
strawberry,5.25,2.5
vanilla,4.75,4


Agora, cada cone possui não apenas seu preço, mas também a classificação de seu sabor.

Em geral, uma chamada para `join` que aumenta uma tabela (digamos `table1`) com informações de outra tabela (digamos `table2`) se parece com isso:

    table1.join(table1_coluna_para_junção, tabela2, tabela2_coluna_para_junção)

A nova tabela `rated` nos permite calcular o preço por estrela, que você pode pensar como uma medida informal de valor. Valores baixos são bons - significam que você está pagando menos por cada estrela de classificação.

In [5]:
rated.with_column('$/Star', rated.column('Price') / rated.column('Stars')).sort(3)

Flavor,Price,Stars,$/Star
vanilla,4.75,4,1.1875
strawberry,3.55,2.5,1.42
chocolate,5.75,3.5,1.64286
chocolate,6.55,3.5,1.87143
strawberry,5.25,2.5,2.1


Embora o morango tenha a classificação mais baixa entre os três sabores, a casquinha de morango mais barata se sai bem nessa medida porque não custa muito por estrela.

**Nota lateral.** A ordem em que listamos as duas tabelas é importante? Vamos tentar. Como você vê, isso altera a ordem em que as colunas aparecem e pode potencialmente alterar a ordem das linhas, mas não não faz nenhuma diferença fundamental.

In [6]:
ratings.join('Kind', cones, 'Flavor')

Kind,Stars,Price
chocolate,3.5,6.55
chocolate,3.5,5.75
strawberry,2.5,3.55
strawberry,2.5,5.25
vanilla,4,4.75


Observe também que a junção conterá apenas informações sobre os itens que aparecem em ambas as tabelas. Vejamos um exemplo. Suponha que haja uma tabela de avaliações de algumas casquinhas de sorvete e tenhamos encontrado a avaliação média para cada sabor.

In [7]:
reviews = Table().with_columns(
    'Flavor', make_array('vanilla', 'chocolate', 'vanilla', 'chocolate'),
    'Stars', make_array(5, 3, 5, 4)
)
reviews

Flavor,Stars
vanilla,5
chocolate,3
vanilla,5
chocolate,4


In [8]:
average_review = reviews.group('Flavor', np.average)
average_review

Flavor,Stars average
chocolate,3.5
vanilla,5


Podemos unir `cones` e `average_review` fornecendo os rótulos das colunas pelas quais unir.

In [9]:
cones.join('Flavor', average_review, 'Flavor')

Flavor,Price,Stars average
chocolate,6.55,3.5
chocolate,5.75,3.5
vanilla,4.75,5


Observe como as casquinhas de morango desapareceram. Nenhuma das avaliações é para casquinhas de morango, então não há nada ao qual as linhas de 'strawberry' possam ser unidas. Isso pode ser um problema, ou pode não ser - isso depende da análise estamos tentando atuar com a tabela unida.